Changing configuration options and their values bear the risk of introducing **configuration errors** due to the violation of their constraints and dependencies. With *CfgNet* we already detect potential dependency conflicts, but miss the violation of constraints. To detect violation of constraints, we analzye file diffs via LLMs to validate the changes made.

*Does configuration changes introduce misconfigurations?*

In [1]:
import json
from typing import Dict, Any
from openai import OpenAI
import os

SYSTEM_PROMPT = """
You are a code review assistant with an excellent knowledge about configuration constraints and dependencies. 
Your task is to analyze the changes in the provided file diffs of a given commit and determine if they introduce any configuration errors.
"""

USER_PROMPT = """
Analyze the following file diffs and validate if the changes introduce any configuration errors:
{commit_diff}

For any potential configuration error, respond in a JSON format as shown below:
{{
"hasError": boolean, // true if there are errors, false if there are none 
"errParameter": [], // List containing properties with errors 
"reason": [], // List containing explanations for each error
}}
"""


def validate_commit_diffs(commit_diff: str) -> Dict:
    """Validate if the changes introduce configuration errors."""
    client = OpenAI(
        api_key=os.getenv("PROXY_SERVER_API_KEY"),
        base_url=os.getenv("BASE_URL")
    )

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT.format(commit_diff=commit_diff)}
    ]

    response = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18", 
        messages=messages,        
        temperature=0.0,
        response_format={"type": "json_object"},
        max_tokens=100,
        timeout=90
    )

    return response.choices[0].message.content

In [ ]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env")

def detect_config_errors(project_data: Dict[str, Any]) -> None:
    """Detect configuration errors in the project data."""
    
    errors = []

    for commit in project_data["commit_data"]:
        if not commit["is_config_related"]:
            continue

        config_files = commit["network_data"]["config_files_data"]

        file_diffs = []

        for config_file in config_files:
            if config_file["is_modified"] and len(config_file["modified_pairs"]) > 0: 
                file_diffs.append(config_file["file_diff"])

        commit_diff = "\n\n".join(file_diffs)

        if not commit_diff:
            continue

        response = validate_commit_diffs(commit_diff=commit_diff)
        detected_error = json.loads(response)
        detected_error["file_path"] = config_file["file_path"]
        detected_error["commit_hash"] = commit["commit_hash"]
        errors.append(detected_error)


    for error in errors:
        print("File Path: ", error["file_path"])
        print("Commit Hash: ", error["commit_hash"])
        print("Has Error: ", error["hasError"])
        print("Error Parameters: ", error["errParameter"])
        print("Reasons: ", error["reason"])
        print()


project_file = "../data/microservice_projects/test-config-repo.json"	
with open(project_file, "r") as f:
    project_data = json.load(f)

detect_config_errors(project_data)


In [ ]:
project_file = "../data/microservice_projects/piggymetrics.json"	
with open(project_file, "r") as f:
    project_data = json.load(f)


for commit in project_data["commit_data"]:
    conflicts = commit["conflicts"]
    if conflicts:
        print("Commit Hash: ", commit["commit_hash"])
        for conflict in conflicts:
            print("Missing Link: ", conflict["link"])
        
    